In [9]:
from pymongo import MongoClient
import numpy as np
import pandas as pd
import time

In [2]:
import ipyparallel as ipp
rc = ipp.Client()
parll=rc[:]

In [3]:
def initParallelClients():
    parll.push({'client':None})
    r=parll.map(initMongos, range(len(parll)))
    r.get();
def initMongos(x):
    global client
    from pymongo import MongoClient
    client = MongoClient()
initParallelClients()

In [5]:
client = MongoClient()
dbRennes = client.congestion

In [6]:
myPoints = list(dbRennes.coyote.find({},{'loc':1,'_id':0}))

In [42]:
def parallelClosestSegment(myPoint):
    res=client.osm.ways.find(
    {   
        "loc":
        {
         "$near": {
            "$geometry": myPoint['loc'],
            "$maxDistance": 20,
         }
       },
        'tag.k':"highway",
        "tag.v": {
          "$in": [
            "motorway", 
            "trunk", 
            "primary",
            "secondary", 
            "tertiary", 
            "motorway_link", 
            "trunk_link", 
            "primary_link",
            "secondary_link", 
            "tertiary_link",
            "unclassified",
            "residential",
            "service"
          ]
        }        
    },{'loc':1,'_id':1}
    ).limit(1)
    return next(res,{'_id':'N/A'})['_id']

In [43]:
startTime = time.time()
segmentIDS=parll.map(parallelClosestSegment,myPoints)
segmentIDS=segmentIDS.get()
endTime = time.time()
print('took {:.3f} ms'.format((endTime-startTime)*1000.0))

took 8710.492 ms


In [ ]:
myPointsDF=pd.DataFrame(list(dbRennes.coyote.find({})))
myPointsDF.matching_road=segmentIDS

In [ ]:
myPointsDF